In [1]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# from deepctr_torch.models import DeepFM
# from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import NFM
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

import torch

/mnt/data/jjw/anaconda3/envs/torch10/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-12 14:46:59.602556: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-12 14:46:59.605344: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 14:46:59.654946: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 14:46:59.656944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performanc

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

读入

In [3]:
test_data = pd.read_csv('data/test/000000000000.csv', sep='\t')
test_data.shape 

(160973, 80)

In [4]:
train_data = pd.read_csv('./data/train.csv', sep='\t')
train_data.shape 

(3485852, 82)

In [5]:
train_data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
0,2541188,57,26325,22294,9018,25604,943,27941,19203,21533,...,2.855607,2.284486,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
1,2541440,60,5156,22294,18971,21545,3448,27941,19606,14659,...,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
2,2541480,65,30256,22294,11104,21545,20366,27941,19203,31372,...,0.571121,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,1,0
3,2541780,63,17216,7152,15742,21545,23877,27941,19606,869,...,0.000000,0.000000,0.000000,0.347077,0.000000,0.0,0.0,0.0,0,0
4,2541833,60,9317,22294,26866,21545,32370,27941,21218,14659,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0


In [6]:
test_data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
0,64505,67,26325,7152,21563,19475,31440,27941,21621,14659,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,64506,67,20095,563,22861,19475,21280,27941,19203,14659,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,64507,67,890,22294,18294,21545,20210,27941,18800,9638,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,64508,67,20095,563,22861,25604,21280,27941,18800,14659,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,64509,67,27426,22294,11338,19475,23855,27941,21218,9638,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [7]:
data = pd.concat([train_data, test_data])

In [8]:
cat_features = ['f_{}'.format(i) for i in range(1, 42)]
bin_features = ['f_{}'.format(i) for i in range(33, 42)]
num_features = ['f_{}'.format(i) for i in range(42, 80)]
date_features = ['f_1']

In [9]:
sparse_features = cat_features
dense_features = num_features

In [10]:
dense_mean = data[dense_features].mean()

In [11]:
data[sparse_features] = data[sparse_features].fillna(-1, )
# data[dense_features] = data[dense_features].fillna(0, )
data[dense_features] = data[dense_features].fillna(dense_mean, )
target = ['is_installed']

In [12]:
data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
0,2541188,57,26325,22294,9018,25604,943,27941,19203,21533,...,2.855607,2.284486,0.115692,1.156922,0.269948,0.0,0.0,0.0,0.0,0.0
1,2541440,60,5156,22294,18971,21545,3448,27941,19606,14659,...,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0.0,0.0
2,2541480,65,30256,22294,11104,21545,20366,27941,19203,31372,...,0.571121,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,1.0,0.0
3,2541780,63,17216,7152,15742,21545,23877,27941,19606,869,...,0.000000,0.000000,0.000000,0.347077,0.000000,0.0,0.0,0.0,0.0,0.0
4,2541833,60,9317,22294,26866,21545,32370,27941,21218,14659,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0.0,0.0


数据处理

In [13]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [14]:
data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
0,2541188,12,114,4,191,4,155,0,1,4,...,2.855607,2.284486,0.115692,1.156922,0.269948,0.0,0.0,0.0,0.0,0.0
1,2541440,15,21,4,390,3,559,0,2,3,...,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0.0,0.0
2,2541480,20,131,4,228,3,3261,0,1,6,...,0.571121,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,1.0,0.0
3,2541780,18,73,2,317,3,3843,0,2,0,...,0.000000,0.000000,0.000000,0.347077,0.000000,0.0,0.0,0.0,0.0,0.0
4,2541833,15,44,4,519,3,5179,0,4,3,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0.0,0.0


In [15]:
# TODO: 不同特征设置不同的Bins

In [16]:
dense_bins = 10

In [17]:
add_sparse_features = []
for fea in tqdm(dense_features, total=len(dense_features)):
    discretizer = KBinsDiscretizer(n_bins=dense_bins, encode='ordinal', strategy='kmeans')  # 等频quantile，等宽uniform
    data[fea + '_encode'] = discretizer.fit_transform(np.array(data[fea].tolist()).reshape(-1, 1))
    add_sparse_features.append(fea + '_encode')

100%|██████████| 38/38 [03:50<00:00,  6.05s/it]


In [18]:
add_sparse_features = [fea + '_encode' for fea in dense_features]

In [19]:
# TODO: 分桶

In [20]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [21]:
data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70_encode,f_71_encode,f_72_encode,f_73_encode,f_74_encode,f_75_encode,f_76_encode,f_77_encode,f_78_encode,f_79_encode
0,2541188,12,114,4,191,4,155,0,1,4,...,2.0,2.0,4.0,4.0,3.0,9.0,7.0,0.0,0.0,0.0
1,2541440,15,21,4,390,3,559,0,2,3,...,1.0,0.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0
2,2541480,20,131,4,228,3,3261,0,1,6,...,3.0,0.0,1.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0
3,2541780,18,73,2,317,3,3843,0,2,0,...,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,2541833,15,44,4,519,3,5179,0,4,3,...,1.0,0.0,0.0,0.0,3.0,9.0,7.0,0.0,0.0,0.0


In [22]:
# data.to_csv('./data/data_bins_10.csv', index=False)

In [23]:
# data = pd.read_csv('./data/data_bins_10.csv')

In [107]:
sparse_dim = 2
dense_dim = 1

In [108]:
# TODO: 去掉dense feature呢，这里没有归一化

In [109]:
# fixlen_feature_columns = \
#     [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=sparse_dim) for i, feat in enumerate(sparse_features)] + \
#     [SparseFeat(feat, vocabulary_size=dense_bins, embedding_dim=sparse_dim) for i, feat in enumerate(add_sparse_features)] + \
#     [DenseFeat(feat, dense_dim) for feat in dense_features]

In [110]:
fixlen_feature_columns = \
    [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=sparse_dim) for i, feat in enumerate(sparse_features)] + \
    [SparseFeat(feat, vocabulary_size=dense_bins, embedding_dim=sparse_dim) for i, feat in enumerate(add_sparse_features)]

In [111]:
fixlen_feature_columns

[SparseFeat(name='f_1', vocabulary_size=23, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_1', group_name='default_group'),
 SparseFeat(name='f_2', vocabulary_size=139, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_2', group_name='default_group'),
 SparseFeat(name='f_3', vocabulary_size=5, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_3', group_name='default_group'),
 SparseFeat(name='f_4', vocabulary_size=638, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_4', group_name='default_group'),
 SparseFeat(name='f_5', vocabulary_size=6, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_5', group_name='default_group'),
 SparseFeat(name='f_6', vocabulary_size=5234, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_6', group_name='default_group'),
 SparseFeat(name='f_7', vocabulary_size=1, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_7', group_name='default_group')

In [112]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [113]:
seed = 2023

单模训练

In [114]:
seed = 2023
device = 'cuda:0'
model_name = 'NFM'
batch_size = 256
epochs = 50
patience = 10

In [115]:
train = data[~data['is_clicked'].isna()]
test = data[data['is_clicked'].isna()]

In [116]:
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [117]:
model = NFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device, seed=seed, dnn_hidden_units=(256, 128))
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'])

In [118]:
model 

NFM(
  (embedding_dict): ModuleDict(
    (f_1): Embedding(23, 2)
    (f_2): Embedding(139, 2)
    (f_3): Embedding(5, 2)
    (f_4): Embedding(638, 2)
    (f_5): Embedding(6, 2)
    (f_6): Embedding(5234, 2)
    (f_7): Embedding(1, 2)
    (f_8): Embedding(6, 2)
    (f_9): Embedding(7, 2)
    (f_10): Embedding(3, 2)
    (f_11): Embedding(24, 2)
    (f_12): Embedding(26, 2)
    (f_13): Embedding(331, 2)
    (f_14): Embedding(19, 2)
    (f_15): Embedding(5854, 2)
    (f_16): Embedding(12, 2)
    (f_17): Embedding(49, 2)
    (f_18): Embedding(924, 2)
    (f_19): Embedding(19, 2)
    (f_20): Embedding(57, 2)
    (f_21): Embedding(35, 2)
    (f_22): Embedding(26, 2)
    (f_23): Embedding(4, 2)
    (f_24): Embedding(4, 2)
    (f_25): Embedding(3, 2)
    (f_26): Embedding(2, 2)
    (f_27): Embedding(2, 2)
    (f_28): Embedding(2, 2)
    (f_29): Embedding(2, 2)
    (f_30): Embedding(3, 2)
    (f_31): Embedding(3, 2)
    (f_32): Embedding(4, 2)
    (f_33): Embedding(2, 2)
    (f_34): Embedding(2,

In [119]:
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=1, patience=patience, mode='min')
mdckpt = ModelCheckpoint(filepath='ckpt/{}_{}_Batch{}_bins{}_onlysparse.ckpt'.format(model_name, sparse_dim, batch_size, dense_bins), monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')
history = model.fit(train_model_input, train_data[target].values, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2, callbacks=[es,mdckpt])

cuda:0
Train on 2788681 samples, validate on 697171 samples, 10894 steps per epoch


10894it [11:05, 16.37it/s]


Epoch 1/50
688s - loss:  0.2963 - binary_crossentropy:  0.2963 - val_binary_crossentropy:  0.2803
Epoch 00001: val_binary_crossentropy improved from inf to 0.28026, saving model to ckpt/NFM_2_Batch256_bins10_onlysparse.ckpt


10894it [11:02, 16.43it/s]


Epoch 2/50
684s - loss:  0.2766 - binary_crossentropy:  0.2765 - val_binary_crossentropy:  0.2763
Epoch 00002: val_binary_crossentropy improved from 0.28026 to 0.27634, saving model to ckpt/NFM_2_Batch256_bins10_onlysparse.ckpt


...

Epoch 27/50
657s - loss:  0.2712 - binary_crossentropy:  0.2702 - val_binary_crossentropy:  0.2743
Epoch 00027: val_binary_crossentropy did not improve from 0.27349


10894it [10:40, 17.00it/s]


Epoch 28/50
665s - loss:  0.2712 - binary_crossentropy:  0.2701 - val_binary_crossentropy:  0.2739
Epoch 00028: val_binary_crossentropy did not improve from 0.27349


10894it [10:30, 17.27it/s]


Epoch 29/50
655s - loss:  0.2712 - binary_crossentropy:  0.2701 - val_binary_crossentropy:  0.2743
Epoch 00029: val_binary_crossentropy did not improve from 0.27349
Epoch 00029: early stopping


单模测试

In [120]:
model = torch.load('ckpt/{}_{}_Batch{}_bins{}_onlysparse.ckpt'.format(model_name, sparse_dim, batch_size, dense_bins))

In [121]:
pred_ans = model.predict(test_model_input, batch_size=batch_size)

In [122]:
pred_ans.min(), pred_ans.max()

(8.274904394056648e-05, 0.9838287830352783)

In [123]:
pred_oof = model.predict(train_model_input, batch_size=batch_size)

In [124]:
logloss = metrics.log_loss(train_data[target], pred_oof)
acc = metrics.roc_auc_score(train_data[target], pred_oof)
precision = metrics.precision_score(train_data[target], [1 if i >= 0.5 else 0 for i in pred_oof])
recall = metrics.recall_score(train_data[target], [1 if i >= 0.5 else 0 for i in pred_oof])
f1 = metrics.f1_score(train_data[target], [1 if i >= 0.5 else 0 for i in pred_oof])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2694, AUC: 0.9018, Precision: 0.7892, Recall: 0.4817, F1 Score: 0.5983


In [125]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = pred_ans
submission.to_csv('./output/{}_{}_{}_{}_{}_Batch{}_bins{}_onlysparse.csv'.format(model_name, seed, len(feature_names), sparse_dim, dense_dim, batch_size, dense_bins), index=False, sep='\t')

交叉验证

In [50]:
K = 5
seed = 2023
device = 'cuda:0'
model_name = 'NFM'
batch_size = 2048
epochs = 50
patience = 10

In [51]:
train = data[~data['is_installed'].isna()]
test = data[data['is_installed'].isna()]
label = train['is_installed'].astype('int')

In [52]:
KF = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
oof_ins = np.zeros(len(train))
predictions_ins = np.zeros((len(test)))
print(len(feature_names))

79


In [55]:
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, label.values)):
    print("fold n°{}".format(fold_))
    
    print(model_name, len(trn_idx), len(val_idx), len(feature_names))
    train_fold = train.iloc[trn_idx]
    train_model_input = {name: train_fold[name] for name in feature_names}
    train_model_label = label.iloc[trn_idx].values
    val_fold = train.iloc[val_idx]
    val_model_input = {name: val_fold[name] for name in feature_names}
    val_model_label = label.iloc[val_idx].values
    test_model_input = {name: test[name] for name in feature_names}

    model = NFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device, seed=seed)
    model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'])
    
    es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=1, patience=patience, mode='min')
    mdckpt = ModelCheckpoint(filepath='ckpt/{}_{}_fold{}.ckpt'.format(model_name, sparse_dim, fold_), 
                             monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')
    history = model.fit(train_model_input, train_model_label, batch_size=batch_size, epochs=epochs, verbose=1, 
                        validation_data=(val_model_input, val_model_label), callbacks=[es, mdckpt])

    model = torch.load('ckpt/{}_{}_fold{}.ckpt'.format(model_name, sparse_dim, fold_))
    oof_ins[val_idx] = model.predict(val_model_input, batch_size=batch_size).flatten()
    predictions_ins[:] += model.predict(test_model_input, batch_size=batch_size).flatten() / K

fold n°0
NFM 2788681 697171 79
cuda:0
Train on 2788681 samples, validate on 697171 samples, 1362 steps per epoch


100%|██████████| 1362/1362 [01:55<00:00, 11.77it/s]


Epoch 1/50
125s - loss:  0.3081 - binary_crossentropy:  0.3081 - val_binary_crossentropy:  0.2799
Epoch 00001: val_binary_crossentropy improved from inf to 0.27988, saving model to ckpt/NFM_16_fold0.ckpt


100%|██████████| 1362/1362 [01:52<00:00, 12.15it/s]


Epoch 2/50
121s - loss:  0.2765 - binary_crossentropy:  0.2765 - val_binary_crossentropy:  0.2741
Epoch 00002: val_binary_crossentropy improved from 0.27988 to 0.27411, saving model to ckpt/NFM_16_fold0.ckpt


...

Epoch 16/50
115s - loss:  0.2472 - binary_crossentropy:  0.2471 - val_binary_crossentropy:  0.2669
Epoch 00016: val_binary_crossentropy did not improve from 0.26427


100%|██████████| 1362/1362 [01:46<00:00, 12.77it/s]


Epoch 17/50
115s - loss:  0.2466 - binary_crossentropy:  0.2465 - val_binary_crossentropy:  0.2669
Epoch 00017: val_binary_crossentropy did not improve from 0.26427


100%|██████████| 1362/1362 [01:46<00:00, 12.74it/s]


Epoch 18/50
115s - loss:  0.2460 - binary_crossentropy:  0.2460 - val_binary_crossentropy:  0.2678
Epoch 00018: val_binary_crossentropy did not improve from 0.26427
Epoch 00018: early stopping


In [56]:
logloss = metrics.log_loss(label, oof_ins)
acc = metrics.roc_auc_score(label, oof_ins)
precision = metrics.precision_score(label, [1 if i >= 0.5 else 0 for i in oof_ins])
recall = metrics.recall_score(label, [1 if i >= 0.5 else 0 for i in oof_ins])
f1 = metrics.f1_score(label, [1 if i >= 0.5 else 0 for i in oof_ins])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2644, AUC: 0.9066, Precision: 0.7778, Recall: 0.5133, F1 Score: 0.6185


In [57]:
predictions_ins.min(), predictions_ins.max()

(5.798138874979486e-06, 0.9576980471611024)

In [58]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_ins
submission.to_csv('./output/{}_{}_{}_{}_{}_Kfold{}.csv'.format(model_name, seed, len(feature_names), sparse_dim, dense_dim, K), index=False, sep='\t')

111